In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import time
import requests
from scipy.stats import linregress

In [2]:
# Import API key
from config import weather_api_key 

# citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File(CSV)
output_data_file = "WeatherPy_output.csv"

# Range of Latitudes and Longitudes
lat_range = (-90, 90)
ing_range = (-180, 180)

SyntaxError: invalid syntax (<ipython-input-2-6a94e2a6a358>, line 5)